In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sliderule import icesat2
import sliderule
import pyproj
from scipy import linalg
import datetime
import geopandas as gpd
import shapely
sliderule.set_verbose(False)
# this is needed in some environments, not in others:
pyproj.datadir.set_data_dir('/home/jovyan/envs/wf_pointCollection/share/proj')
%matplotlib widget

# The workflow:

This is a quick-and-dirty representation of how we would map height-change rates for a small region on the ice sheet.  The workflow could be scaled up to make a map of elevation-change rate for an ice sheet with the caveats that the data volume increases dramatically toward the poles and that fitting a single elevation-change rate does not take full advantage of the data.

Workflow steps:

1. Read all available ATL06 data for a 40x40-km box centered on x0, y0
2. Filter the data for atl06_quality_summary==0
3. Select data in each 1x1 km box in the dataset, fit the selected data with a vertically moving plane

The remaining cells in the notebook plot the results


In [ ]:
%%time
xy0 = [0., -3.0e6]
crs=3413
W=4.e4
pad=np.array([-W/2, W/2])
bbox = [xy0[0] + pad[[0, 1, 1, 0, 0]], xy0[1] + pad[[0, 0, 1, 1, 0]]]
# generate a clipping polygon in a lat/lon coordinate system
bbox_ll = pyproj.Transformer.from_crs(pyproj.CRS(crs), 
                                      pyproj.CRS(4326))\
                    .transform(*bbox)
poly = [{"lat":lat, "lon":lon} for lat, lon in zip(*bbox_ll)]

save_file = f'ATL06_E{int(xy0[0]/1.e3)}_N{int(xy0[1]/1.e3)}.parquet'
icesat2.atl06sp(
   {
    "poly": poly,
    "t0":"2018-10-13T00:00:00Z",
    "t1":"2025-10-13T00:00:00Z", 
   
    "output": {
      "format": "parquet",
      "as_geo": True,
      "path": save_file,
      "with_checksum": False
    }})
df=gpd.read_parquet(save_file).to_crs(3413)

# set a decimal-year time field, filter the data for quality

In [ ]:
df['time'] =(np.array(df.index) - np.datetime64(datetime.datetime(2018, 1, 1))).astype(float)/1.e9/24/3600/365.25 +2018
df=df[df.atl06_quality_summary==0]

In [ ]:
# Iterate over 1-km tiles in the data

In [ ]:
def make_index_dict(data, bin_size):
    xy0 = np.round(np.array(data.geometry.x+1j*data.geometry.y)/bin_size)*bin_size
    u_ii, index, inverse=np.unique(xy0, return_index=True, return_inverse=True)
    uXY=xy0[index]
    
    bin_dict={}
    inv_arg_ind=np.argsort(inverse)
    inv_sort=inverse[inv_arg_ind]
    ind_delta=np.concatenate([[-1], np.where(np.diff(inv_sort))[0], [inv_sort.size]])
    for ii in range(len(ind_delta)-1):
        this_ind=inv_arg_ind[(ind_delta[ii]+1):(ind_delta[ii+1]+1)]
        this_xy0 = xy0[this_ind[0]]
        bin_dict[tuple([np.real(this_xy0), np.imag(this_xy0)])]=this_ind
    return bin_dict

In [ ]:
# make a function that fits a time-varying plane to the data for a selected set of points

def dhdt_est(x, y, z, t, ind):
    #xx=np.array(D.geometry.x[ind])
    #yy=np.array(D.geometry.y[ind])
    #zz=np.array(D.h_li[ind])
    #tt =np.array(D.time[ind])
    xx=x[ind]
    yy=y[ind]
    zz=z[ind]
    tt=t[ind]
    G=np.c_[np.ones(len(ind)), tt-2020, xx-np.mean(xx), yy-np.mean(yy)]
    sigma = 1.e4
    keep = np.ones(G.shape[0], dtype=bool)
    keep_last = np.zeros_like(keep)
    n_iterations=0
    try:
        # iterate fit to reduce outliers
        while not np.all(keep_last==keep) and n_iterations  < 5:
            n_iterations += 1
            m=linalg.lstsq(G[keep,:], zz[keep])[0]
            r=zz-G@m
            sigma_r = np.nanstd(r[keep])
            keep = np.abs(r) < 3*np.maximum(sigma, 0.1)
        # estimate errors
        G1 = G[keep,:]
        Ginv = np.linalg.inv(G1.T@G1)@G1.T
        Cm = Ginv@Ginv.T*(np.maximum(0.03**2, sigma_r**2))
        sigma_dhdt = np.sqrt(Cm[1][1])
        return m[1], np.sum(keep), sigma_r, sigma_dhdt
    except np.linalg.LinAlgError:
        return np.nan, np.nan, np.nan, np.nan


In [ ]:
%time
# apply the dh/dt function to each 1- km square in the data
# my initial test of this passed the dataframe to the dhdt_est function, and extracted a subset from (e.g.) df.geometry.x
# for each bin.  This was terribly slow.  Extracting x, y, z, and t to numpy arrays outside the loop made the results much faster.

bin_dict=make_index_dict(df, 1.e3)
xy_dhdt = []
x=np.array(df.geometry.x)
y=np.array(df.geometry.y)
z=np.array(df.h_li)
t=np.array(df.time)
for xyc, ind in bin_dict.items():
    dhdt, N, sigma_r, sigma_dhdt = dhdt_est(x, y, z, t, ind)
    xy_dhdt += [{'geometry':shapely.Point(xyc),
                 'dhdt':dhdt,
                 'N':N,'sigma_r':sigma_r,
                 'sigma_dhdt':sigma_dhdt}]
    
D_dhdt=gpd.GeoDataFrame(xy_dhdt)

In [ ]:
# plot the results:

hf, hax=plt.subplots(1, 2, figsize=[8,4], layout='constrained', sharex=True, sharey=True)

plt.sca(hax[0])
small_σ= D_dhdt.sigma_dhdt < 0.125
plt.plot(D_dhdt.geometry.x[small_σ==0], D_dhdt.geometry.y[small_σ==0],'.', color='darkgray')
plt.colorbar(
    plt.scatter(D_dhdt.geometry.x[small_σ], D_dhdt.geometry.y[small_σ], 3, c=D_dhdt.dhdt[small_σ], cmap='RdBu', clim=[-1, 1]), 
    label='dh/dt, m/yr', shrink=0.5, extend='both')
plt.gca().set_facecolor('lightgray')
plt.gca().set_title('dh/dt')


plt.sca(hax[1])
plt.colorbar(
    plt.scatter(D_dhdt.geometry.x, D_dhdt.geometry.y, 3, c=D_dhdt.sigma_dhdt, cmap='magma', clim=[0, 1]), 
    label='dh/dt uncertainty, m/yr', shrink=0.5, extend='max')
plt.gca().set_facecolor('lightgray')
plt.gca().set_title('uncertainty')

for ax in hax: 
    ax.set_aspect(1)

# Speed from test run:

For xy0 = [0, -3.e6] in Greenland,
* Fetching data: 28.8 s
* Reading cache: 456 ms
* Processing: 5 microseconds 
